In [22]:
import pandas as pd
import requests
import datetime

link = "https://bet2.hkjc.com/marksix/getJSON.aspx?"

date_from = datetime.date(2010,11,9)
date_to = datetime.datetime.now().date()

print("from:", date_from)
print("to:", date_to)

import os
print(os.getcwd())


def endmonth(d):
    y = d.year
    m = d.month
    if m == 12:
        y += 1
        m = 1
    else:
        m += 1
    return datetime.date(y,m,1) - datetime.timedelta(days=1)

date_current = date_from


total_record = pd.DataFrame()

s = requests.session()

while date_current <= date_to:
    sd = date_current.strftime("%Y%m%d")
    ed = min(endmonth(date_current), date_to)
    date_current = ed + datetime.timedelta(days=1)
    ed = ed.strftime("%Y%m%d")
    
    link2 = link+f"sd={sd}&ed={ed}&sb=0"
    r = s.get(link2)
    if r.content == b'\r\n':
        continue
    record_df = pd.DataFrame(r.json())
    record_df["no"] = record_df["no"].str.split("+")
    no_df = pd.DataFrame(record_df["no"].to_list(),columns=["no_"+str(i) for i in range(1,7)])
    record_df.drop(columns=["no"],inplace=True)
    record_df = pd.concat([record_df, no_df],axis=1)
    record_df = record_df[[col for col in record_df.columns if col != "sno"] + ["sno"]]
    
    total_record = pd.concat([total_record, record_df])

    
for col in ["p"+str(i) for i in range(1, 8)] + ["inv","sno"] + ["p"+str(i)+"u" for i in range(1, 8)] + ["no_"+str(i) for i in range(1,7)]:
    total_record[col] = total_record[col].str.replace(",","").astype(float)
    
total_record["date"] = pd.to_datetime(total_record["date"],format=("%d/%m/%Y")).dt.date
total_record.sort_values(by=["date"],inplace=True)
total_record.reset_index(inplace=True,drop=True)

from: 2010-11-09
to: 2025-06-16
C:\Users\lenovo\python jupyter notebooks\web scraping\mark six data


In [23]:
display(total_record)
total_record.to_excel(f"mark_six(raw) ({date_from:%Y%m%d}-{date_to:%Y%m%d}).xlsx",index=False)

cols = ["date","inv"]+[f"p{i}u" for i in range(4,8)]+[f"no_{i}" for i in range(1,7)]+["sno"]
total_record[cols].to_excel(f"mark_six_data ({date_from}~{date_to}).xlsx",index=False)

,id,date,sbcode,sbnameE,sbnameC,inv,p1,p1u,p2,p2u,...,p6u,p7,p7u,no_1,no_2,no_3,no_4,no_5,no_6,sno
0,10/130,2010-11-09,ANN,Mark Six 35 Anniversary Snowball,六合彩35周年金多寶,75939693.0,0.0,0.0,1325170.0,3.0,...,6644.0,40.0,104882.7,4.0,15.0,41.0,43.0,45.0,49.0,18.0
1,10/131,2010-11-11,,,,66989785.0,46496730.0,0.5,3190040.0,1.0,...,6925.5,40.0,108945.1,5.0,6.0,15.0,28.0,29.0,43.0,48.0
2,10/132,2010-11-13,,,,45195821.0,0.0,0.0,1963760.0,1.0,...,5932.2,40.0,79578.6,7.0,18.0,21.0,28.0,47.0,48.0,35.0
3,10/133,2010-11-16,,,,56396898.0,37212370.0,0.5,1345450.0,2.0,...,7259.5,40.0,81594.8,3.0,12.0,20.0,39.0,46.0,49.0,19.0
4,10/134,2010-11-18,,,,41264389.0,0.0,0.0,854070.0,2.5,...,4237.5,40.0,56362.2,6.0,20.0,30.0,33.0,39.0,47.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,25/061,2025-06-03,,,,38810229.0,8000000.0,1.0,0.0,0.0,...,4197.6,40.0,53785.6,1.0,2.0,13.0,15.0,41.0,46.0,27.0
1993,25/062,2025-06-05,,,,40409117.0,0.0,0.0,469820.0,3.0,...,6015.9,40.0,67981.4,18.0,22.0,24.0,33.0,39.0,44.0,3.0
1994,25/063,2025-06-07,,,,43565282.0,12932520.0,1.0,1096110.0,1.5,...,5655.6,40.0,64562.6,9.0,15.0,27.0,30.0,34.0,39.0,19.0
1995,25/064,2025-06-10,,,,40913202.0,0.0,0.0,1589040.0,0.5,...,4111.0,40.0,55658.8,9.0,13.0,35.0,40.0,45.0,46.0,26.0


In [8]:
print(date_from)

2010-11-09
